In [ ]:
import os
import numpy as np
import cv2
import time

weights = os.path.join('yolo', 'yolov3.weights')
cfg = os.path.join('yolo', 'yolov3.cfg')
coco = os.path.join('yolo', 'coco.names')
net = cv2.dnn.readNet(weights, cfg)
classes = []
with open(coco, 'r') as f:
    classes = f.read().splitlines()
cap = cv2.VideoCapture(0)
save_directory = "captures" 
while True:
    ret, frame = cap.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layer_outputs = net.forward(output_layers_names)
    
    person_detected = False
    weapon_detected = False
    person_box = None
    weapon_box = None

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                label = str(classes[class_id])
                if label == "person":
                    person_detected = True
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    person_box = [x, y, w, h]

                if label in ["knife", "gun"]:
                    weapon_detected = True
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    weapon_box = [x, y, w, h]

    if person_detected:
        x, y, w, h = person_box
        color = (0, 255, 255)  
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, "person", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    if weapon_detected:
        x, y, w, h = weapon_box
        color = (0, 0, 255) 
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, "weapon", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    if person_detected and weapon_detected:
        save_frame_path = os.path.join(save_directory, f"frame_{time.time()}.jpg")
        cv2.imwrite(save_frame_path, frame)
        print(f"Frame saved: {save_frame_path}")

    cv2.imshow("Object Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()